In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import warnings
warnings.filterwarnings("ignore")

# 🎨 Renk paleti
custom_palette = ['#7ab288', '#f8f6f0', '#f9bb23', '#d82359', '#f38fb1', '#ffcdd3', '#fafafa', '#ffffff', '#8d8a8a', '#000000']

# 📂 Dosyaları oku
df_questions = pd.read_excel('15-18 ay sorular.xlsx')
df = pd.read_csv('generated_children_data_300.csv')

question_columns = [col for col in df.columns if col.startswith("Q")]
skill_areas = ['Sosyal', 'Duyusal', 'Motor', 'Dil', 'Ortak_Dikkat', 'İletişim']
disorder_areas = ['Otizm', 'Disleksi', 'DEHB', 'Dil ve Konuşma Bozukluğu', 'Gelişim Eksikliği ve Koordinasy', 'Zihinsel Yetersizlik']


In [2]:
# 📊 Başlangıç dağılımı

def get_distribution(df_subset, label_list, title):
    dist = pd.DataFrame({
        'Etiket': label_list,
        'Sayı': [df_subset[label].sum() for label in label_list]
    })
    dist['Yüzde'] = round(dist['Sayı'] / len(df_subset) * 100, 2)
    dist['Grup'] = title
    return dist

dist_all = pd.concat([
    get_distribution(df, disorder_areas, 'Bozukluk (Tümü)'),
    get_distribution(df, skill_areas, 'Gelişimsel Eksiklik (Tümü)')
])

# 📊 Test verisi dağılımı (50 çocuk)
df_train, df_test = train_test_split(df, test_size=50, stratify=df['Any_Disorder'], random_state=42)
X_train_full = df_train[question_columns]
y_train = df_train['Any_Disorder']
X_test_full = df_test[question_columns]
y_test = df_test['Any_Disorder']

dist_test = pd.concat([
    get_distribution(df_test, disorder_areas, 'Bozukluk (Test)'),
    get_distribution(df_test, skill_areas, 'Gelişimsel Eksiklik (Test)')
])

In [3]:
param_grid_models = {
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=1000, solver='liblinear', random_state=42),
        "params": {
            "C": [0.01, 0.1, 1, 10, 100],
            "penalty": ["l1", "l2", None]
        }  # 5 × 3 = 15 kombinasyon
    },
    "Decision Tree": {
        "model": DecisionTreeClassifier(random_state=42),
        "params": {
            "criterion": ["gini", "entropy"],
            "max_depth": [2, 3, 5, 10, None],
            "min_samples_split": [2, 5, 10, 20],
            'min_samples_leaf': [1, 2, 4]
        }  # 2 × 5 × 4 x 3 = 120 kombinasyon
    },
    "Random Forest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200, 400, 800],
            "max_depth": [3, 5, 10, 20, None],
            "min_samples_split": [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }  # 5 × 5 × 3 x 3 = 225 kombinasyon
    },
    "SVM": {
        "model": SVC(probability=True, random_state=42),
        "params": {
            "C": [0.01, 0.1, 1, 10, 100],
            "kernel": ["linear", "rbf", "poly"],
            'gamma': [1, 0.1, 0.01, 0.001],
        }  # 5 × 3 × 4 = 60 kombinasyon
    },
    "KNN": {
        "model": KNeighborsClassifier(),
        "params": {
            "n_neighbors": [3, 5, 7, 9, 11, 13, 15],
            "weights": ["uniform", "distance"],
            "metric": ["euclidean", "manhattan"]
        }  # 7 x 2 × 2  = 28 kombinasyon
    },
    "XGBoost": {
        "model": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 5, 7],
            "learning_rate": [0.01, 0.05, 0.1],
            "subsample": [0.5, 0.8, 1.0]
        }  # 3 × 3 × 3 × 3 = 81 kombinasyon
    },
    "LightGBM": {
        "model": lgb.LGBMClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 5, 7],
            "learning_rate": [0.01, 0.05, 0.1],
            "num_leaves": [15, 31]
        }  # 3 × 3 × 3 × 2 = 54 kombinasyon
    },
    "CatBoost": {
        "model": cb.CatBoostClassifier(verbose=0, random_state=42),
        "params": {
            "iterations": [50, 100, 200],
            "depth": [3, 5, 7],
            "learning_rate": [0.01, 0.05, 0.1],
            "l2_leaf_reg": [1, 3]
        }  # 3 × 3 × 3 × 2 = 54 kombinasyon
    }
}


In [4]:
# ⚙️ Model bazlı feature importance ile değerlendirme fonksiyonu
import openpyxl
import time
import os
import traceback
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import joblib  # ✅ model kayıt için eklendi

output_file = "model_ozet_sonuclar.xlsx"
wb = Workbook()
wb.remove(wb.active)  # varsayılan boş sayfayı sil
model_outputs = []  # model sonuçlarını topla

# 🔁 Model eğitimi ve sonuç çıkarımı ayrı fonksiyon
def run_gridsearch_and_select_features(name, model_obj, param_grid, X_train, y_train, X_test, y_test, k=30):
    print(f"{name} modeli için grid search başlatılıyor...")
    try:
        grid = GridSearchCV(model_obj, param_grid, cv=5, scoring='f1', n_jobs=-1)
        grid.fit(X_train, y_train)
        best_full = grid.best_estimator_
        y_pred_full = best_full.predict(X_test)
        f1_full = f1_score(y_test, y_pred_full)

        # Özellik önemi hesaplama - SVM ve KNN için alternatif yöntem
        if hasattr(best_full, 'feature_importances_'):
            importances = best_full.feature_importances_
        elif hasattr(best_full, 'coef_'):
            importances = np.abs(best_full.coef_).flatten()
        else:
            print(f"{name} için standart özellik önemi bulunamadı, permütasyon önemi kullanılıyor...")
            perm_importance = permutation_importance(best_full, X_train, y_train,
                                                    n_repeats=10, random_state=42, n_jobs=-1)
            importances = perm_importance.importances_mean

        # En önemli k özellik
        indices = np.argsort(importances)[::-1][:k]
        selected_columns = X_train.columns[indices]

        # Seçilmiş özelliklerle tekrar model eğit
        grid_selected = GridSearchCV(model_obj, param_grid, cv=5, scoring='f1', n_jobs=-1)
        grid_selected.fit(X_train[selected_columns], y_train)
        best_selected = grid_selected.best_estimator_
        y_pred_selected = best_selected.predict(X_test[selected_columns])
        f1_selected = f1_score(y_test, y_pred_selected)

        print(f"{name} modeli tamamlandı. F1 (Full): {f1_full:.4f}, F1 (Selected): {f1_selected:.4f}")

        # ✅ Modeli kayıt et
        model_path = f"model_exports/models/{name.replace(' ', '')}.pkl"
        if os.path.exists(model_path):
            os.remove(model_path)

        joblib.dump(best_selected, model_path)

        return {
            "Model": name,
            "F1_Full": f1_full,
            "F1_Selected": f1_selected,
            "Selected_Columns": selected_columns,
            "Y_True": y_test,
            "Y_Pred_Full": y_pred_full,
            "Y_Pred_Selected": y_pred_selected
        }

    except Exception as e:
        print(f"{name} modeli eğitilirken hata: {e}")
        traceback.print_exc()
        return None


In [5]:
# 🔁 Tüm modelleri eğit
model_results = []
for name, obj in param_grid_models.items():
    result = run_gridsearch_and_select_features(
        name,
        obj['model'],
        obj['params'],
        X_train_full,
        y_train,
        X_test_full,
        y_test
    )
    if result:
        model_results.append(result)

Logistic Regression modeli için grid search başlatılıyor...
Logistic Regression modeli tamamlandı. F1 (Full): 0.9429, F1 (Selected): 0.8857
Decision Tree modeli için grid search başlatılıyor...
Decision Tree modeli tamamlandı. F1 (Full): 0.8406, F1 (Selected): 0.8824
Random Forest modeli için grid search başlatılıyor...
Random Forest modeli tamamlandı. F1 (Full): 0.9706, F1 (Selected): 0.8406
Random Forest modeli eğitilirken hata: [Errno 2] No such file or directory: 'model_exports/models/RandomForest.pkl'
SVM modeli için grid search başlatılıyor...


Traceback (most recent call last):
  File "C:\Users\ruvey\AppData\Local\Temp\ipykernel_11256\419726539.py", line 60, in run_gridsearch_and_select_features
    joblib.dump(best_selected, model_path)
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\numpy_pickle.py", line 552, in dump
    with open(filename, 'wb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'model_exports/models/RandomForest.pkl'


SVM için standart özellik önemi bulunamadı, permütasyon önemi kullanılıyor...
SVM modeli tamamlandı. F1 (Full): 1.0000, F1 (Selected): 0.7952
SVM modeli eğitilirken hata: [Errno 2] No such file or directory: 'model_exports/models/SVM.pkl'
KNN modeli için grid search başlatılıyor...


Traceback (most recent call last):
  File "C:\Users\ruvey\AppData\Local\Temp\ipykernel_11256\419726539.py", line 60, in run_gridsearch_and_select_features
    joblib.dump(best_selected, model_path)
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\numpy_pickle.py", line 552, in dump
    with open(filename, 'wb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'model_exports/models/SVM.pkl'


KNN için standart özellik önemi bulunamadı, permütasyon önemi kullanılıyor...
KNN modeli tamamlandı. F1 (Full): 0.9524, F1 (Selected): 0.8197
KNN modeli eğitilirken hata: [Errno 2] No such file or directory: 'model_exports/models/KNN.pkl'
XGBoost modeli için grid search başlatılıyor...


...

Traceback (most recent call last):
  File "C:\Users\ruvey\AppData\Local\Temp\ipykernel_11256\419726539.py", line 60, in run_gridsearch_and_select_features
    joblib.dump(best_selected, model_path)
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\numpy_pickle.py", line 552, in dump
    with open(filename, 'wb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'model_exports/models/XGBoost.pkl'
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\ruvey\AppData\Local\Progr

[LightGBM] [Info] Number of positive: 165, number of negative: 85
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 502
[LightGBM] [Info] Number of data points in the train set: 250, number of used features: 251
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.660000 -> initscore=0.663294
[LightGBM] [Info] Start training from score 0.663294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

Traceback (most recent call last):
  File "C:\Users\ruvey\AppData\Local\Temp\ipykernel_11256\419726539.py", line 60, in run_gridsearch_and_select_features
    joblib.dump(best_selected, model_path)
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\numpy_pickle.py", line 552, in dump
    with open(filename, 'wb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'model_exports/models/LightGBM.pkl'


CatBoost modeli tamamlandı. F1 (Full): 0.9697, F1 (Selected): 0.8955
CatBoost modeli eğitilirken hata: [Errno 2] No such file or directory: 'model_exports/models/CatBoost.pkl'


Traceback (most recent call last):
  File "C:\Users\ruvey\AppData\Local\Temp\ipykernel_11256\419726539.py", line 60, in run_gridsearch_and_select_features
    joblib.dump(best_selected, model_path)
  File "C:\Users\ruvey\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\numpy_pickle.py", line 552, in dump
    with open(filename, 'wb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'model_exports/models/CatBoost.pkl'


In [6]:
# 📊 Colab İçin Model Sonuçlarını Excel'e Yazan Fonksiyon
import io
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.utils.dataframe import dataframe_to_rows
from sklearn.metrics import confusion_matrix

def export_results_to_excel(results, output_file, df_questions=None, skill_columns_map=None): # Added df_questions and skill_columns_map as parameters
    print(f"Excel'e yazılacak model sayısı: {len(results)}")
    print(f"Model isimleri: {[res['Model'] for res in results]}")
    # Yeni Excel workbook oluştur
    wb = Workbook()
    if wb.active:
        wb.remove(wb.active)  # Varsayılan sayfayı kaldır

    overview_data = []

    for res in results:
        name = res['Model']
        f1_full = res['F1_Full']
        f1_selected = res['F1_Selected']
        y_test = res['Y_True']
        y_pred_full = res['Y_Pred_Full']
        y_pred_selected = res['Y_Pred_Selected']
        selected_columns = res['Selected_Columns']

        # Görselleştirme hazırla
        fig, axes = plt.subplots(1, 2, figsize=(10, 4))
        sns.heatmap(confusion_matrix(y_test, y_pred_full), annot=True, fmt='d', cmap='YlGnBu', ax=axes[0])
        axes[0].set_title(f"{name} - Full")
        sns.heatmap(confusion_matrix(y_test, y_pred_selected), annot=True, fmt='d', cmap='YlOrRd', ax=axes[1])
        axes[1].set_title(f"{name} - Selected")
        fig.tight_layout()

        # Görsel dosyasını kaydet
        img_path = f"{name.lower().replace(' ', '_')[:40]}_conf.png"
        fig.savefig(img_path, dpi=150)
        plt.close(fig)

        # Excel sayfası oluştur
        ws = wb.create_sheet(title=name[:30])
        ws.append(["Model Adı", name])
        ws.append(["F1 Skoru (Full)", f1_full])
        ws.append(["F1 Skoru (Seçilmiş)", f1_selected])
        ws.append([])
        ws.append(["Seçilen En Önemli Sorular"])
        for col in selected_columns:
            ws.append([col])

        # Görseli Excel'e ekle (Colab'de çalışacak şekilde)
        if os.path.exists(img_path):
            try:
                img_excel = ExcelImage(img_path)
                img_excel.width = 420
                img_excel.height = 180
                ws.add_image(img_excel, "G2")
            except Exception as e:
                ws.append(["Uyarı", f"Görsel eklenemedi: {e}"])
                print(f"Görsel eklenemedi: {e}")
        else:
            ws.append(["Uyarı", "Confusion Matrix görseli bulunamadı."])

        # Karşılaştırma için veri ekle
        overview_data.append({
            'Model': name,
            'F1_Full': f1_full,
            'F1_Selected': f1_selected
        })

    # Genel karşılaştırma sayfası
    overview = pd.DataFrame(overview_data)
    ws_all = wb.create_sheet(title="Karşılaştırma")
    for r in dataframe_to_rows(overview, index=False, header=True):
        ws_all.append(r)

    # Dosyayı kaydet
    try:
        wb.save(output_file)
        print(f"Sonuçlar başarıyla Excel dosyasına kaydedildi: {output_file}")

        # Colab'de dosyayı indirmek için
        files.download(output_file)
        print("Excel dosyası indirildi!")

        # Ayrıca model görselleri de indirilsin
        for res in results:
            name = res['Model']
            img_path = f"{name.lower().replace(' ', '_')[:40]}_conf.png"
            if os.path.exists(img_path):
                files.download(img_path)
                print(f"{img_path} görseli indirildi!")
    except Exception as e:
        print(f"Excel dosyası kaydedilemedi/indirilemedi: {e}")

# Dağılım görselleştirmesi ve Excel'e ekleme
def create_distribution_excel(dist_all, output_file="distribution_results.xlsx"):
    # Görselleştirme
    custom_palette = sns.color_palette(["#7ab288", "#f9bb23", "#d82359", "#f38fb1", "#ffcdd3", "#8d8a8a"])

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    sns.barplot(data=dist_all[dist_all['Grup'] == 'Bozukluk (Tümü)'], x='Etiket', y='Sayı', ax=axes[0], palette=custom_palette)
    axes[0].set_title('Toplam Bozukluk Dağılımı')
    axes[0].tick_params(axis='x', rotation=45)

    sns.barplot(data=dist_all[dist_all['Grup'] == 'Gelişimsel Eksiklik (Tümü)'], x='Etiket', y='Sayı', ax=axes[1], palette=custom_palette)
    axes[1].set_title('Toplam Eksiklik Dağılımı')
    axes[1].tick_params(axis='x', rotation=45)

    fig.tight_layout()

    # Görseli kaydet
    dist_img_path = "genel_dagilimlar.png"
    fig.savefig(dist_img_path, dpi=150)
    plt.close(fig)

    # Excel dosyası oluştur
    wb = Workbook()
    if wb.active:
        wb.remove(wb.active)

    ws = wb.create_sheet("Başlangıç Dağılımı")

    # Görsel ekle
    if os.path.exists(dist_img_path):
        try:
            img = ExcelImage(dist_img_path)
            img.width = 700
            img.height = 300
            ws.add_image(img, "B2")
        except Exception as e:
            ws.append(["Uyarı", f"Görsel eklenemedi: {e}"])
            print(f"Görsel eklenemedi: {e}")

    # Dağılım verilerini ekle
    ws.append([])
    ws.append([])
    ws.append([])
    ws.append([])
    ws.append([])
    ws.append([])
    ws.append([])
    ws.append([])
    ws.append([])
    ws.append([])  # Görselden sonra biraz boşluk bırak

    for r in dataframe_to_rows(dist_all, index=False, header=True):
        ws.append(r)

    # Kaydet ve indir
    try:
        wb.save(output_file)
        print(f"Dağılım verileri Excel dosyasına kaydedildi: {output_file}")

        # Colab'de dosyayı indirmek için
        files.download(output_file)
        print("Dağılım Excel dosyası indirildi!")

        # Görseli de indir
        if os.path.exists(dist_img_path):
            files.download(dist_img_path)
            print("Dağılım görseli indirildi!")
    except Exception as e:
        print(f"Excel dosyası kaydedilemedi/indirilemedi: {e}")

# Colab'de çalıştırma örneği
# export_results_to_excel(model_results, "model_sonuclar.xlsx")
# create_distribution_excel(dist_all, "dagilim_sonuclar.xlsx")

In [7]:
export_results_to_excel(model_results, "model_sonuclar.xlsx")


Excel'e yazılacak model sayısı: 2
Model isimleri: ['Logistic Regression', 'Decision Tree']
Sonuçlar başarıyla Excel dosyasına kaydedildi: model_sonuclar.xlsx
Excel dosyası kaydedilemedi/indirilemedi: name 'files' is not defined


In [8]:
# Dağılım verilerini Excel'e aktar ve indir
create_distribution_excel(dist_all, "dagilim_sonuclar.xlsx")

Dağılım verileri Excel dosyasına kaydedildi: dagilim_sonuclar.xlsx
Excel dosyası kaydedilemedi/indirilemedi: name 'files' is not defined


In [9]:
import numpy as np
import joblib
import json
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

# Sütun haritası
skill_columns_map = {
    'Sosyal': 'Sosyal Beceri',
    'Duyusal': 'Duyusal Beceri',
    'Motor': 'Motor beceri',
    'Dil': 'Dil Becerisi',
    'OrtakDikkat': 'Ortak Dikkat Becerileri',
    'İletişim': 'İletişim Becerileri'
}

# Dummy veri üretimi (100 özellikten oluşan rastgele veri)
np.random.seed(42)
X_dummy = pd.DataFrame(np.random.randint(0, 2, size=(100, 100)), columns=[f"Q{i}" for i in range(1, 101)])
y_dummy = np.random.randint(0, 2, size=100)

# Kullanılacak modeller (isimler boşluksuz, app.py ile uyumlu)
model_list = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "KNN": KNeighborsClassifier()
}

# Klasörleri oluştur
os.makedirs("model_exports/models", exist_ok=True)
model_sorular_dict = {}

# Her model için rastgele 30 soru seçerek model eğitimi ve kayıt
for name, model in model_list.items():
    model.fit(X_dummy, y_dummy)
    model_path = f"model_exports/models/{name}.pkl"
    joblib.dump(model, model_path)

    selected_indices = np.random.choice(range(len(df_questions)), size=30, replace=False)
    selected_indices.sort()
    selected_codes = [f"Q{i+1}" for i in selected_indices]
    selected_texts = df_questions.iloc[selected_indices]["Soru"].tolist()
    selected_skills = [
        [skill for skill, col in skill_columns_map.items()
         if str(df_questions.iloc[i][col]).strip().lower() == "evet"]
        for i in selected_indices
    ]

    model_sorular_dict[name] = {
        "questions": selected_codes,
        "texts": selected_texts,
        "skills": selected_skills
    }

# JSON dosyasını yaz
with open("model_exports/model_sorular.json", "w", encoding="utf-8") as f:
    json.dump(model_sorular_dict, f, ensure_ascii=False, indent=4)

list(model_sorular_dict.keys())



['RandomForest',
 'LogisticRegression',
 'DecisionTree',
 'GradientBoosting',
 'SVM',
 'KNN']

In [10]:
import joblib
import os

model_folder = "model_exports/models"

for file in os.listdir(model_folder):
    if file.endswith(".pkl"):
        model_path = os.path.join(model_folder, file)
        try:
            model = joblib.load(model_path)
            print(f"{file}: ✅ {type(model)}")
        except Exception as e:
            print(f"{file}: ❌ HATA — {e}")


DecisionTree.pkl: ✅ <class 'sklearn.tree._classes.DecisionTreeClassifier'>
GradientBoosting.pkl: ✅ <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
KNN.pkl: ✅ <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
LogisticRegression.pkl: ✅ <class 'sklearn.linear_model._logistic.LogisticRegression'>
RandomForest.pkl: ✅ <class 'sklearn.ensemble._forest.RandomForestClassifier'>
SVM.pkl: ✅ <class 'sklearn.svm._classes.SVC'>
